In [ ]:
!python -V


Python 3.10.12


In [2]:
import os
import torch
import cv2
import numpy as np
import pandas as pd
from os import walk
from os.path import join
from datetime import datetime
import matplotlib.pyplot as plt
%matplotlib inline
# connect to your google drive
from google.colab import drive
from google.colab.patches import cv2_imshow
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
# clone git repositories in workspace
!git clone https://github.com/WongKinYiu/yolov9.git

# change directory to yolov7 file
%cd /content/yolov9

Cloning into 'yolov9'...
remote: Enumerating objects: 621, done.
remote: Counting objects: 100% (238/238), done.
remote: Compressing objects: 100% (116/116), done.
remote: Total 621 (delta 186), reused 122 (delta 122), pack-reused 383
Receiving objects: 100% (621/621), 3.21 MiB | 7.98 MiB/s, done.
Resolving deltas: 100% (238/238), done.
/content/yolov9


In [4]:
!unzip /content/gdrive/MyDrive/HW1_material.zip -d /content/yolov9

Archive:  /content/gdrive/MyDrive/HW1_material.zip
   creating: /content/yolov9/HW1_material/
   creating: /content/yolov9/HW1_material/detect/
  inflating: /content/yolov9/HW1_material/detect/parking_area.png  
   creating: /content/yolov9/HW1_material/datasets/
   creating: /content/yolov9/HW1_material/datasets/hw1/
   creating: /content/yolov9/HW1_material/datasets/hw1/images/
   creating: /content/yolov9/HW1_material/datasets/hw1/images/train/
  inflating: /content/yolov9/HW1_material/datasets/hw1/images/train/tc_1.png  
  inflating: /content/yolov9/HW1_material/datasets/hw1/images/train/tc_2.png  
  inflating: /content/yolov9/HW1_material/datasets/hw1/images/train/tc_3.png  
  inflating: /content/yolov9/HW1_material/datasets/hw1/images/train/tc_4.png  
  inflating: /content/yolov9/HW1_material/datasets/hw1/images/train/tc_5.png  
  inflating: /content/yolov9/HW1_material/datasets/hw1/images/train/tc_6.png  
  inflating: /content/yolov9/HW1_material/datasets/hw1/images/train/tc_7.p

###  Train yolov9 model

In [6]:
# Finetune the Yolov7 model
'''
You can add other parameters by using --params_name params_values ex: --device 0 --epoch 1 --batch-size 1
just do not change parameters that we already give you.
hint: only one instruction
'''
'''
Free to load another YoloV7 model that you can find here: https://github.com/WongKinYiu/yolov7
hint: only one instruction

The best model will save in /content/yolov7/runs/train/yolov7-tiny/weights.
If you run this cell again then the model will save if /content/yolov7/runs/train/yolov7-tiny2/weight and so on.
'''
# !python train.py --img-size 360 160  --epoch 100 --batch-size 8 --data HW1_material/hw1.yaml --weights 'yolov7-tiny-custom' --exist-ok --cfg cfg/training/yolov7-tiny.yaml --name yolov7-tiny --hyp data/hyp.scratch.custom.yaml
!python train_dual.py --device 0 --batch 16 --img-size 360 --data HW1_material/hw1.yaml --cfg models/detect/yolov9-c.yaml --weights '' --name yolov9-c --hyp hyp.scratch-high.yaml --min-items 0 --epochs 100 --close-mosaic 15

2024-03-24 15:40:28.583132: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-24 15:40:28.583184: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-24 15:40:28.584618: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-24 15:40:29.741410: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
train_dual: weights=, cfg=models/detect/yolov9-c.yaml, data=HW1_material/hw1.yaml, hyp=hyp.scratch-high.yaml, epochs=100, batch_size=16, imgsz=360, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolv

In [8]:
'''Save model in your google drive place'''
savePath = '/content/gdrive/MyDrive/best5.pt'
!cp  '/content/yolov9/runs/train/yolov9-c2/weights/best.pt' $savePath

In [20]:
# If you don't have detect/train just ignore this remind (rm: cannot remove '/content/yolov7/runs/detect/train': No such file or directory)
!rm -r '/content/yolov9/runs/detect/train'

# Apply on training data
'''
Run evaluation like cell above
you must use parameter: --source /content/yolov7/HW1_material/train/car --name train --save-txt --exist-ok --weights /content/gdrive/MyDrive/ai/hw1/best.pt
hint: one instruction starting from !python
'''
!python detect_dual.py --device 0 --source /content/yolov9/HW1_material/train/car --name train --save-txt --exist-ok --weights $savePath
'''
Run evaluation like cell above
you must use parameter: --source /content/yolov7/HW1_material/train/non-car --name train --save-txt --exist-ok --weights $savePath
hint: one instruction starting from !python
'''

!python detect_dual.py --device 0 --source /content/yolov9/HW1_material/train/non-car --name train --save-txt --exist-ok --weights $savePath


detect_dual: weights=['/content/gdrive/MyDrive/best5.pt'], source=/content/yolov9/HW1_material/train/car, data=data/coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=0, view_img=False, save_txt=True, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=train, exist_ok=True, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLO 🚀 v0.1-76-g519346b Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
yolov9-c summary: 604 layers, 50698278 parameters, 0 gradients
image 1/300 /content/yolov9/HW1_material/train/car/tc_1.png: 288x640 1 parking, 110.8ms
image 2/300 /content/yolov9/HW1_material/train/car/tc_10.png: 288x640 2 parkings, 52.9ms
image 3/300 /content/yolov9/HW1_material/train/car/tc_100.png: 288x640 (no detections), 52.8ms
image 4/300 /content/yolov9/HW1_material/train/car/tc_101.p

In [21]:
def Calculate(AnswerFile,PredFile):

  def loadAns(dataPath):
      pos = os.path.join(dataPath, 'car')
      neg = os.path.join(dataPath, 'non-car')
      Answer = []
      for f in os.listdir(pos):
          if not os.path.isdir(f):
              Answer.append((os.path.join(pos, f), 1))
      for f in os.listdir(neg):
          if not os.path.isdir(f):
              Answer.append((os.path.join(neg, f), 0))
      return Answer

  Answer = loadAns(AnswerFile)

  all_negatives, all_positives = 0, 0
  true_negatives, false_negatives = 0, 0
  true_positives, false_positives = 0, 0

  for x, y in Answer:

      exec_path = "'" + x + "'"
      img_filename = x.split('/')[-1]
      img_filename = img_filename.split('.')[0]
      txtpath = PredFile + img_filename + '.txt'

      if y == 1:
          all_positives += 1
      else:
          all_negatives += 1

      try:
          with open(txtpath, 'r') as f:
              n = len(f.readlines())
          if n > 0:
              prediction = 1
          else:
              prediction = 0
      except:
          prediction = 0

      if prediction == 1:
          if y == 1:
              true_positives += 1
          else:
              false_positives += 1
      else:
          if y == 1:
              false_negatives += 1
          else:
              true_negatives += 1

  correct = true_positives + true_negatives
  print("False Positive Rate: %d/%d (%f)" % (false_positives, all_negatives, false_positives/all_negatives))
  print("False Negative Rate: %d/%d (%f)" % (false_negatives, all_positives, false_negatives/all_positives))
  print("Training Accuracy: %d/%d (%f)" % (correct, len(Answer), correct/len(Answer)))

In [25]:
# Calculate yolov7 performance
# You have to adjust the parameters to get more than 90% accuracy
# Warning: make sure that txtpath is currect because you may get many train{n} file when training more than one time.

Calculate('/content/yolov9/HW1_material/train/','/content/yolov9/runs/detect/train/labels/')

False Positive Rate: 12/300 (0.040000)
False Negative Rate: 27/300 (0.090000)
Training Accuracy: 561/600 (0.935000)


In [23]:
# If you don't have detect/test just ignore this remind (rm: cannot remove '/content/yolov7/runs/detect/test': No such file or directory)
!rm -r '/content/yolov9/runs/detect/test'

# Apply on testing data
'''
Run evaluation like cell above
you must use parameter: --source /content/yolov7/HW1_material/test/car --name test --save-txt --exist-ok --weights $savePath
hint: one instruction starting from !python
'''
!python detect_dual.py --source /content/yolov9/HW1_material/test/car --name test --save-txt --exist-ok --weights $savePath

'''
Run evaluation like cell above
you must use parameter: --source /content/yolov7/HW1_material/test/non-car --name test --save-txt --exist-ok --weights $savePath
hint: one instruction starting from !python
'''
!python detect_dual.py --source /content/yolov9/HW1_material/test/non-car --name test --save-txt --exist-ok --weights $savePath

rm: cannot remove '/content/yolov9/runs/detect/test': No such file or directory
detect_dual: weights=['/content/gdrive/MyDrive/best5.pt'], source=/content/yolov9/HW1_material/test/car, data=data/coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=True, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=test, exist_ok=True, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLO 🚀 v0.1-76-g519346b Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
yolov9-c summary: 604 layers, 50698278 parameters, 0 gradients
image 1/300 /content/yolov9/HW1_material/test/car/vc_1.png: 288x640 2 parkings, 107.2ms
image 2/300 /content/yolov9/HW1_material/test/car/vc_10.png: 288x640 1 parking, 54.1ms
image 3/300 /content/yolov9/HW1_material/test/car/vc_100.png: 288x640 2 parking

In [24]:
Calculate('/content/yolov9/HW1_material/test/','/content/yolov9/runs/detect/test/labels/')

False Positive Rate: 4/300 (0.013333)
False Negative Rate: 14/300 (0.046667)
Training Accuracy: 582/600 (0.970000)
